[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/H-IAAC/d3vis_ipynb/blob/main/examples/dashboard.ipynb)

In [ ]:
from d3vis_ipynb import DecisionPlot, WaterfallPlot, MatrixLayout, BeeswarmPlot, ForcePlot, Button, ScatterPlot
from sklearn.manifold import TSNE
import xgboost
import shap
import ssl
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True)
ssl._create_default_https_context = ssl._create_unverified_context
# train XGBoost model
X, y = shap.datasets.adult()
X_display, y_display = shap.datasets.adult(display=True)
model = xgboost.XGBClassifier(n_estimators=100, max_depth=2).fit(X, y)

# compute SHAP values
explainer = shap.Explainer(model, X)
shap_values = explainer(X[:1000])

In [ ]:
X.head()

In [ ]:
y

In [ ]:
beeswarmPlot = BeeswarmPlot(explanation=shap_values[:500])
waterfallPlot = WaterfallPlot(explanation=shap_values[0])
button = Button(description="Calculate TSNE")
scatterplot = ScatterPlot(data=pd.DataFrame(), x='dim1', y='dim2', hue='color')

def on_beeswarm_selected(values):
    if(beeswarmPlot.selectedValues):
        waterfallPlot.explanation = beeswarmPlot.selectedValues[0]

beeswarmPlot.on_select_values(on_beeswarm_selected)

def calculate_TSNE(values):
    result = np.sum(beeswarmPlot.selectedValues.values, axis=1) + beeswarmPlot.selectedValues[0].base_values
    result = result >= 0
    tsne = TSNE(n_components=2, random_state=42)
    data_2d = tsne.fit_transform(beeswarmPlot.selectedValues.values)
    df = pd.DataFrame({
        'dim1': data_2d[:, 0],
        'dim2': data_2d[:, 1],
        'color': result
    })
    scatterplot.data = df
    
button.on_click(calculate_TSNE)

matrix = [[1, 1, 1, 5],
          [1, 1, 1, 5],
          [1, 1, 1, 5],
          [2, 3, 3, 3],
          [4, 3, 3, 3]]
matrixLayout = MatrixLayout(matrix)
matrixLayout.add(beeswarmPlot, 1)
matrixLayout.add(waterfallPlot, 5)
matrixLayout.add(button, 2)
matrixLayout.add(scatterplot, 3)
matrixLayout

In [ ]:
scatterplot.selectedValues